# Query Embedding


In [ ]:

from dotenv import load_dotenv
import os
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import AzureSearch

load_dotenv()

embedding_api_key = os.getenv('Embedding_API_KEY')
embedding_endpoint = os.getenv('Embedding_ENDPOINT')

embedding_api_version = os.getenv('embedding_api_version')
embedding_deployment = os.getenv('embedding_deployment')

os.environ.pop("OPENAI_API_BASE", None)
os.environ.pop("BASE_URL", None)

ai_search_endpoint = os.getenv('AI_SEARCH_ENDPOINT')
ai_search_index_name = os.getenv('AI_SEARCH_INDEX_NAME')
ai_search_api_key = os.getenv('AI_SEARCH_API_KEY')



In [7]:
print("실행디렉토리:", os.getcwd())
print("Embedding_API_KEY:", os.getenv("Embedding_API_KEY"))
print(ai_search_endpoint)

실행디렉토리: c:\Users\blank\OneDrive\바탕 화면\2차 프로젝트 레포\홍원님 모델
Embedding_API_KEY: 11UsWjDAHdmUd88UAtmuBciucqMX0L9Qafi0Mzg0a9Pw5hIl90QkJQQJ99BDACfhMk5XJ3w3AAABACOGsig4
https://6a021-aisearch.search.windows.net


In [2]:

import os
import uuid
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import AzureSearch
embedding_api_version = "2024-02-15-preview"
embedding_deployment = "text-embedding-3-small"
os.environ.pop("OPENAI_API_BASE", None)
os.environ.pop("BASE_URL", None)

embedding = AzureOpenAIEmbeddings(
    api_key=embedding_api_key,
    azure_endpoint=embedding_endpoint,
    model=embedding_deployment,
    openai_api_version=embedding_api_version
)

### Bringing Source Results

In [5]:
import requests

def request_ai_search(query: str, source_filter: str = None, k: int = 5) -> list:
    headers = {
        "Content-Type": "application/json",
        "api-key": ai_search_api_key
    }

    query_vector = embedding.embed_query(query)

    body = {
        "search": query,  # 🔥 키워드 검색 추가!
        "vectorQueries": [
            {
                "kind": "vector",
                "vector": query_vector,
                "fields": "embedding",
                "k": k
            }
        ]
    }

    if source_filter:
        cleaned_source = source_filter.replace(".pdf", "")
        body["filter"] = f"source eq '{cleaned_source}'"

    response = requests.post(ai_search_endpoint, headers=headers, json=body)

    if response.status_code != 200:
        print(f"❌ 검색 실패: {response.status_code}")
        print(response.text)
        return []

    response_json = response.json()
    results = [
        {
            "content": item["content"],
            "source": item.get("source", ""),
            "score": item.get("@search.score", 0)
        }
        for item in response_json["value"]
    ]

    return results


# 예시 실행
query = "신청 기간 알려줘"
source = "merged_붙임1. (25.03.28.)청년매입임대 입주자 모집공고문_게시용"  # 확장자는 제거되어 비교됨

results = request_ai_search(query,source_filter=source, k=5)

for i, result in enumerate(results):
    print(f"\n🔍 Top {i+1} (score: {result['score']:.4f})")
    print(f"출처: {result['source']}")
    print(result["content"])



🔍 Top 1 (score: 0.0317)
출처: merged_붙임1. (25.03.28.)청년매입임대 입주자 모집공고문_게시용
신청자, 부모 주민등록표초본은 이혼한 부모가 신청자의 주민등록등본상 모두 미등재된 경우, 신청자와 모집공고일 현재 실질적 부양의무를 이행하거나 신청자가 심사 대상으로 선택한 부 또는 모의 초본을 제출해야 합니다. 초본은 과거 주소변동사항, 세대주 성명 및 관계 등이 전부 표기되도록 발급되어야 하며, 발급처는 행정복지센터입니다.
자산보유사실확인서는 2순위 신청 시 본인 및 부모를 포함하여 작성하며, 3순위 신청 시 본인만 작성해야 합니다. 임대차계약서는 반드시 확정일자를 받은 것으로 제출해야 하며, 작성자는 신청자이고 양식이 첨부됩니다.
금융정보등제공동의서는 2순위 신청 시 본인 및 부모를 포함하여 작성하며, 3순위 신청 시 본인만 작성해야 합니다. 동의서 미제출 시 신청과 접수가 거부되며, 반드시 정자 서명이 필요합니다. 작성자는 신청자이고 양식이 첨부됩니다.

🔍 Top 2 (score: 0.0306)
출처: merged_붙임1. (25.03.28.)청년매입임대 입주자 모집공고문_게시용
공급단계는 청약신청 단계에서 시작됩니다. 청약신청은 1순위는 25년 4월 7일 월요일 오전 10시부터 오후 5시까지, 2, 3순위는 25년 4월 8일 화요일 오전 10시부터 4월 9일 수요일 오후 5시까지 진행됩니다. 청년 매입임대는 인터넷 접수를 원칙으로 하며, 모바일 청약은 불가합니다. 신청자는 공고 내용을 숙지한 후 신청자격, 소득 등 입력사항을 확인하고 신청해야 합니다. 청약신청 기간 중에는 24시간 신청이 가능하지만, 시작일과 마감일을 제외합니다. 청약 시 공동인증서 또는 민간인증서를 사전에 발급받아야 하며, 1인 1모집단위 신청만 가능하고 중복 신청 시 모든 신청이 무효 처리됩니다. 또한, 입주순위를 잘못 신청하면 접수신청 기한 후 순위 변경이 불가하며 탈락 처리됩니다.  
서류제출 대상자 발표는 25년 4월 18일 금요일 오후 5시 이

### Connecting with LLM

In [ ]:
import requests
import re

# Azure OpenAI GPT-4o 설정
llm_endpoint = os.getenv('OPENAI_ENDPOINT')
llm_api_key = os.getenv('OPENAI_API_KEY')

def request_gpt(prompt: str) -> str:
    """
    GPT-4o에 프롬프트를 보내고 응답 받기
    """
    headers = {
        'Content-Type': 'application/json',
        'api-key': llm_api_key
    }

    body = {
        "messages": [
            {
                "role": "system",
                "content": "너는 친절하고 정확한 AI 도우미야. 사용자 질문에 문서 기반으로 답해줘."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 1024
    }

    response = requests.post(llm_endpoint, headers=headers, json=body)

    if response.status_code == 200:
        response_json = response.json()
        message = response_json['choices'][0]['message']
        content = message['content']
        # [doc1] 같은 포맷이 있을 경우 사람이 읽기 쉽게 변환
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        return content
    else:
        print("❌ 요청 실패:", response.status_code, response.text)
        return "⚠️ 오류가 발생했습니다."


In [11]:
def generate_answer_with_rag(query: str, source_filter: str = None, top_k: int = 3) -> str:
    # 🔍 벡터 유사도 기반 검색 수행 (REST API 기반)
    results = request_ai_search(query, source_filter=source_filter, k=top_k)

    # 결과가 없으면 안내
    if not results:
        return "❌ 관련 문서를 찾을 수 없습니다."

    # 🔗 context 구성
    context = "\n\n".join([f"[doc{i+1}]\n{item['content']}" for i, item in enumerate(results)])

    # 🤖 LLM 프롬프트 생성
    prompt = f"""다음은 사용자가 질문한 내용과 관련된 문서 내용이야. 이 문서를 참고해서 질문에 대해 정확하고 구체적으로 답변해줘.
                그리고 어떤 문서에서 찾았는지도 출처도 알려줘
[사용자 질문]
{query}

[참고 문서]
{context}

답변:"""

    return request_gpt(prompt)



In [ ]:
query = ""
source = "merged_아츠스테이영등포_입주자모집공고문"
answer_gpt = request_gpt(query)
print("🤖 LLM 응답:\n", answer_gpt)
answer = generate_answer_with_rag(query, source_filter=None, top_k=5)
print("🧠 LLM + RAG 응답:\n", answer)


🤖 GPT 응답:
 어떤 신청 기간에 대해 알고 싶으신지 구체적으로 말씀해 주시면, 더욱 정확하고 유용한 정보를 제공해 드릴 수 있습니다! 예를 들어, 대학 입학 원서, 장학금, 공공 지원 프로그램 등 특정 대상이나 분야를 알려주세요. 😊
🧠 GPT 응답:
 신청 기간은 다음과 같습니다:

- **신청 접수 기간**: 2025년 3월 31일부터 4월 6일까지. 우편 접수는 소인분까지 유효하며, 현장 또는 우편 접수로 진행됩니다.
- **보완 서류 접수 기간**: 2025년 4월 9일부터 4월 11일까지. 이는 실제 서류 도착일까지 유효하며, 신청서류 접수 기간 내 접수자만 해당됩니다.

출처: [참조 33]
